**Group 22**

Name  | Surname | Email  
---------|-------------------|---------
Julio|Vigueras|20220661@novaims.unl.pt 
Ariel|Pérez|20220662@novaims.unl.pt
Miguelanguel|Mayuare|20220665@novaims.unl.pt
Ayotunde|Aribo|20221012@novaims.unl.pt

# Model handcrafted "E"
---

This model implements BatchNormalization instead of rescaling and resizing the images. The convolutional layers have padding set as "same" and kernel initializer set as GlorotNormal.
Each block consists on two convolutional layers. The output of the first convolutional layer is passed to the activation function and then to the next convolutional layer. This second convolutional layer passes its output to the Batch Normalization layer and then it is passed to the activation function. Thereafter, a max pooling layer with strides equal to 2 is applied, and it finishes with a dropout layer.
After iterating over the number of blocks one desire, it applies a Convolutional layer, a Batch Normalization layer, an activation function, flattens the output, a Dense layer and finishes giving the output probabilities with the softmax function. 



In [ ]:
# Make the imports
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory

import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import pathlib

# from google.colab import drive

Load the database and unzip it

In [ ]:
!wget https://www.dropbox.com/s/n3320qxwdn3rs19/moths.zip?dl=0 -O moths.zip
!unzip -q moths.zip

--2023-04-08 18:45:39--  https://www.dropbox.com/s/n3320qxwdn3rs19/moths.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/n3320qxwdn3rs19/moths.zip [following]
--2023-04-08 18:45:40--  https://www.dropbox.com/s/raw/n3320qxwdn3rs19/moths.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc76593929104d2c41a5249e4f0a.dl.dropboxusercontent.com/cd/0/inline/B5wuIogU_wQXDtTgb8Ycz2AUwn4qknhSCAZbHdlSZL3BCuZqWla0mnu40gk5_kMLaU5BZ0BPLgRmMN72Y922P2bRhEj48XgvlgN1Yjgabl2qcwyIefOyCBOSikWl6bO7eTM-DU5Ga2tnjKCli6HViQ_bgdpzCe6VF6OUg8aA18_ePg/file# [following]
--2023-04-08 18:45:40--  https://uc76593929104d2c41a5249e4f0a.dl.dropboxusercontent.com/cd/0/inline/B5wuIogU_wQXDtTgb8Ycz2AUwn4qknhSCAZbHdlSZL3BCuZqWla0mnu40gk5_kMLaU5BZ0BPLgRmMN72Y

In [ ]:
def model_E(blocks=4, input_shape=(224, 224, 3)):
    data_augmentation = keras.Sequential([
        layers.RandomRotation(0.2),
        layers.RandomFlip(),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
        layers.RandomZoom(0.1),
    ])

    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    for i in range(blocks + 1, blocks + 5):
        x = layers.Conv2D(filters=2**i, kernel_size=3,
                           kernel_initializer=initializers.GlorotNormal(seed=123), activation="relu")(x)
        x = layers.Conv2D(filters=2**i, kernel_size=3, use_bias=False,
                          kernel_initializer=initializers.GlorotNormal(seed=123))(x)
        # Add BatchNormalization instead of Rescaling and Resizing
        x = layers.BatchNormalization()(x) 
        x = layers.Activation("relu")(x)
        x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
        x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, use_bias=False)(x)
    # Add BatchNormalization instead of Rescaling and Resizin
    x = layers.BatchNormalization()(x) g
    x = layers.Activation("relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(50, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = model_E(3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 16)      2304      
                                                                 
 batch_normalization (BatchN  (None, 220, 220, 16)     64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 220, 220, 16)      0         
                                                             

In [ ]:
# Compile model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
                metrics=["accuracy"])

In [ ]:
dataset_path = pathlib.Path("moths")
input_shape = (224, 224, 3)
batch_size = 64

In [ ]:
# Split datasets
train_dataset = image_dataset_from_directory(
    dataset_path / "train",
    image_size=input_shape[:2],
    batch_size=batch_size)
validation_dataset = image_dataset_from_directory(
    dataset_path / "valid",
    image_size=input_shape[:2],
    batch_size=batch_size)
test_dataset = image_dataset_from_directory(
    dataset_path / "test",
    image_size=input_shape[:2],
    batch_size=batch_size)

Found 3558 files belonging to 30 classes.
Found 445 files belonging to 30 classes.
Found 408 files belonging to 30 classes.


In [ ]:
# Create callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="moths/saved_models/model_handcrafted_E.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [ ]:
# Train model
history = model.fit(
    train_dataset,
    epochs=100,
    batch_size=64,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/100
56/56 [==============================] - 36s 295ms/step - loss: 3.8042 - accuracy: 0.0995 - val_loss: 3.5547 - val_accuracy: 0.0989
Epoch 2/100
56/56 [==============================] - 13s 230ms/step - loss: 2.6605 - accuracy: 0.2119 - val_loss: 3.2728 - val_accuracy: 0.1843
Epoch 3/100
56/56 [==============================] - 13s 229ms/step - loss: 2.2333 - accuracy: 0.3184 - val_loss: 3.8825 - val_accuracy: 0.1910
Epoch 4/100
56/56 [==============================] - 14s 236ms/step - loss: 1.9812 - accuracy: 0.3800 - val_loss: 2.4692 - val_accuracy: 0.3393
Epoch 5/100
56/56 [==============================] - 13s 230ms/step - loss: 1.8502 - accuracy: 0.4185 - val_loss: 2.6275 - val_accuracy: 0.3124
Epoch 6/100
56/56 [==============================] - 14s 235ms/step - loss: 1.7209 - accuracy: 0.4587 - val_loss: 2.3302 - val_accuracy: 0.3483
Epoch 7/100
56/56 [==============================] - 14s 238ms/step - loss: 1.5747 - accuracy: 0.4989 - val_loss: 2.0455 - val_accuracy:

In [ ]:
# Visualization
hist_df = pd.DataFrame(history.history)
loss = px.scatter(hist_df['loss'])
val_loss = px.line(hist_df['val_loss'])
accuracy = px.scatter(hist_df['accuracy'])
val_accuracy = px.line(hist_df['val_accuracy'])

fig = make_subplots(cols=2, rows=1, subplot_titles=("Loss", "Accuracy"))
fig.add_trace(loss.data[0], col=1, row=1)
fig.add_trace(val_loss.data[0], col=1, row=1)
fig.add_trace(accuracy.data[0], col=2, row=1)
fig.add_trace(val_accuracy.data[0], col=2, row=1)

fig.show()

---
It can be seen that the model improves quickly its accuracy during the first third of epochs. Then it continues to improve but at a slower pace. It seems that it does not present overfitting during its training, at least until the last epochs, where the gap between the accuracy of the model on the training data and the accuracy on the validation data widens. Our inference regarding the good results obtained with this model lies, firstly, in the data augmentation, since it increases the training data and tries to cover the variance that real data could present; secondly, in the Batch Normalization because, as it is well known, it reduce the internal covariate shift of the data; thirdly, The Max Pooling layer with a size equals to 2 and strides equals to 2 simplifies the data in a more treatable manner; lastly, the dropout helps the neurons to learn and generalize more features, trying to avoid overfitting.